## Preprocessament

In [180]:
import pandas as pd
import numpy as np

Lectura de dades

In [181]:
activitats_v = pd.read_csv("data/raw/activitats_valencia.csv")
activitats_a = pd.read_csv("data/raw/activitats_alacant.csv")
activitats_c = pd.read_csv("data/raw/activitats_castello.csv")
ajuntaments_info = pd.read_csv("data/raw/ajuntaments_info_raw.csv")

In [182]:
activitats_v.columns

Index(['Marca de temps', 'Localitat', 'Data', 'Categoria',
       'En cas d'altres, quina?', 'Nom de l'activitat ', 'Companyia, artista',
       'Lloc', 'Llengua de l'activitat', 'Preu', 'Regidoria organitzadora',
       'Llengua de la publicitat', 'Font',
       'Persona que ha introduït les dades', 'Unnamed: 14', 'Unnamed: 15',
       'Estructura dades', 'Observacions estructura dades'],
      dtype='object')

In [183]:
ajuntaments_info.columns

Index(['MUNICIPI', 'PROVÍNCIA', 'COMARCA', 'N HABITANTS',
       'ÌNDEX ENVELLIMENT POBLACIÓ (2021)',
       'Municipis que tenen reconeguda la condició de municipis turístics',
       'PARTIT GOVERNANT EN 2022-23', 'PARTIT GOVERNANT 2023-24',
       'PARTIT GOVERNANT 2024-25',
       'Subvenció normalització\n de l'us del valencià \nper part de la Generalitat (2022)',
       'LLENGUA PREDOMINANT', 'Regidoria Cultura 2022',
       'Regidoria Cultura 2023', 'REGIDORIA CULTURA 2025',
       'Consulta llengua 2025 Valencià', 'Consulta llengua 2025 Castellà'],
      dtype='object')

* Seleccio de variables i canvi de nom

Marca temporal,Localitat,Data,Categoria,"En cas d'altres, quina?",Nom de l'activitat ,"Companyia, artista",Lloc,Llengua de l'activitat,Preu,Regidoria organitzadora,Llengua de la publicitat,Font,Persona que ha introduït les dades

val:
Marca de temps,Localitat,Data,Categoria,"En cas d'altres, quina?","Nom de l'activitat ","Companyia, artista",Lloc,Llengua de l'activitat,Preu,Regidoria organitzadora,Llengua de la publicitat,Font,Persona que ha introduït les dades,,,Estructura dades,Observacions estructura dades


In [184]:
columns = ["Marca temporal", "Localitat", "Data", "Categoria", "En cas d'altres, quina?", "Nom de l'activitat ", "Companyia, artista", "Lloc", "Llengua de l'activitat", "Preu", "Regidoria organitzadora", "Llengua de la publicitat", "Font",  "Persona que ha introduït les dades"]
columns_v = ["Marca de temps", "Localitat", "Data", "Categoria", "En cas d'altres, quina?", "Nom de l'activitat ", "Companyia, artista", "Lloc", "Llengua de l'activitat", "Preu", "Regidoria organitzadora", "Llengua de la publicitat", "Font",  "Persona que ha introduït les dades"]
column_rename = ["marca_temps", "localitat", "data", "categoria", "categoria_alternativa", "nom_activitat", "companyia", "lloc", "llengua_activitat", "preu", "regidoria", "llengua_publicitat", "font", "persona_dades"]
columns_info_rename = ["ajuntament", "provincia", "comarca", "n_habitants", "index_envelliment_2021", "pob_turistica", "govern_22_23", "govern_23_24", "govern_24_25",  "subv_us_valencia_2022", "predomini_ling", "regidoria_cultura", "regidoria_cultura_2023", "regidoria_cultura_2025", "consulta_llengua_2025_valencia",  "consulta_llengua_2025_castella"]

In [185]:
activitats_v

,Marca de temps,Localitat,Data,Categoria,"En cas d'altres, quina?",Nom de l'activitat,"Companyia, artista",Lloc,Llengua de l'activitat,Preu,Regidoria organitzadora,Llengua de la publicitat,Font,Persona que ha introduït les dades,Unnamed: 14,Unnamed: 15,Estructura dades,Observacions estructura dades
0,06/02/2022 12:08:37,Canals,07/10/2022,Música,NaN,XIX Campanya de concerts d'intercanvis 2022,Unió Musical Santa Cecília,Centre Cultural Papa Calixte III,Valencià/català,NaN,En col·laboració o organitzat per una entitat ...,Bilingüe,https://www.facebook.com/AjuntamentCanals?loca...,Joan,NaN,NaN,NaN,NaN
1,06/02/2022 12:10:51,Canals,30/10/2022,Exposició,NaN,EXPOSICIÓ CARTELLS DEL CLUB DE LECTURA,Josep Moltó,Centre Cultural Ca Don José.,Activitat sense llengua,NaN,Cultura,Valencià/català,https://www.facebook.com/AjuntamentCanals?loca...,Joan,NaN,NaN,NaN,NaN
2,06/02/2022 12:14:43,Canals,08/10/2022,Música,NaN,XIX Campanya de concerts d'intercanvis 2022,Cor Calixtí de Canals amb l'orquestra de Pols ...,Centre Cultural Papa Calixte III,Espanyol,NaN,En col·laboració o organitzat per una entitat ...,Bilingüe,https://www.facebook.com/AjuntamentCanals?loca...,Joan,NaN,NaN,NaN,NaN
3,06/02/2022 12:17:18,Canals,09/10/2022,Música,NaN,SISENA EDICIÓ DEL CORRELLENGUA,NaN,parc Jaume I,Valencià/català,NaN,Cultura,Valencià/català,https://www.facebook.com/AjuntamentCanals?loca...,Joan,NaN,NaN,NaN,NaN
4,06/02/2022 12:21:57,Canals,11/10/2022,Taller de teatre,NaN,Teatre i Prevenció de Drogues,NaN,NaN,No hi ha informació,NaN,Joventut,Valencià/català,https://www.facebook.com/AjuntamentCanals?loca...,Joan,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9863,31/03/2025 10:21:47,València,09/03/2025,Altres,Taller artístic,Ecofeminisme,Meu Turó,CC La Reina,Valencià/català,NaN,Cultura,Bilingüe,https://www.instagram.com/culturalvalencia/,Gabriel,NaN,NaN,NaN,NaN
9864,31/03/2025 10:22:13,València,07/03/2025,Exposició,NaN,La Dula: exposició col·lectiva elaborada per l...,NaN,NaN,Valencià/català,NaN,Cultura,Bilingüe,https://www.instagram.com/culturalvalencia/,Osel,NaN,NaN,NaN,NaN
9865,31/03/2025 10:22:53,València,09/03/2025,Dansa,NaN,Cuerpos Imposibles,Isabel Is My Name,CC La Reina,Activitat sense llengua,NaN,Cultura,Bilingüe,https://www.instagram.com/culturalvalencia/,Gabriel,NaN,NaN,NaN,NaN
9866,31/03/2025 10:25:04,València,05/03/2025,Exposició,NaN,Si pudiera elegir mi paisaje,NaN,CC del Carme,Activitat sense llengua,NaN,Cultura,Valencià/català,https://www.instagram.com/p/DG00gc2CD2C/,Neus,NaN,NaN,NaN,NaN


In [186]:
activitats_v = activitats_v[columns_v]
activitats_v.columns = column_rename
activitats_c = activitats_c[columns_v]
activitats_c.columns = column_rename
activitats_a = activitats_a[columns_v]
activitats_a.columns = column_rename
ajuntaments_info.columns = columns_info_rename

* Juntem els 3 datasets per a conformar el dataset activitats

In [187]:
activitats = pd.concat([activitats_a, activitats_v, activitats_c], axis = 0)

In [188]:
activitats.shape

(21293, 14)

## Preprocessament de dades

* Tranformacions en variables

**ajuntaments_info**

pob_turistica

In [189]:
# Adegim el valor "No" a aquelles poblacions que falten dades
ajuntaments_info["pob_turistica"].loc[ajuntaments_info["pob_turistica"].isnull()] = "No"

C:\Users\quimm\AppData\Local\Temp\ipykernel_9412\1527247745.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ajuntaments_info["pob_turistica"].loc[ajuntaments_info["pob_turistica"].isnull()] = "No"
C:\Users\quimm\AppData\Local\Temp\ipyker

index_envelliment_2021

In [190]:
ajuntaments_info["index_envelliment_2021"] = pd.to_numeric(ajuntaments_info["index_envelliment_2021"].str.replace(",", ".") )

subv_us_valencia_2022

In [191]:
ajuntaments_info["subv_us_valencia_2022"] = pd.to_numeric(ajuntaments_info["subv_us_valencia_2022"].str.replace(".", "").str.replace("€", "").str.replace(",", ".") )

consulta_base


In [192]:
ajuntaments_info["consulta_llengua_2025_valencia"] = ajuntaments_info["consulta_llengua_2025_valencia"].str.replace("%", "")
ajuntaments_info["consulta_llengua_2025_castella"] = ajuntaments_info["consulta_llengua_2025_castella"].str.replace("%", "")

**activitats**

llengua_activitat

In [193]:
def assign_value_more_than_2_languages(llengua_activitat):
    """
    si la variable té dos llengues o més, el valor canvia a "Dos o més llengües"
    """
    llengua_activitat_split = llengua_activitat.split(", ")

    if len(llengua_activitat_split) > 1:
        if "Activitat sense llengua" not in llengua_activitat_split and "No hi ha informació" not in llengua_activitat_split:
            return "Dos o més llengües"
    
    return llengua_activitat

In [194]:
activitats = activitats.loc[~ activitats["llengua_activitat"].isna()]

In [195]:
activitats["llengua_activitat"] = activitats["llengua_activitat"].apply(lambda x: assign_value_more_than_2_languages(x))

localitat

In [196]:
activitats = activitats.loc[activitats["localitat"] != "Bilingüe"]
activitats = activitats.loc[activitats["localitat"] != "Monforte del Cid"]

data

In [197]:
def is_date_valid(date):
    try:
        pd.to_datetime(date, format= "%d/%m/%Y")
        return True
    except:
        return False

In [198]:
activitats = activitats.loc[activitats.data.apply(is_date_valid)]

In [199]:
# activitats = activitats.loc[~pd.to_datetime(activitats.data, format= "%d/%m/%Y").dt.month.isin([4,5])]

Variables de text

In [200]:
activitats.nom_activitat=activitats.nom_activitat.str.replace(",", ";")
activitats.nom_activitat = activitats.nom_activitat.str.replace("\n", "")
activitats.nom_activitat = activitats.nom_activitat.str.replace("\r", "")
activitats.companyia = activitats.companyia.str.replace(",", ";")
activitats.lloc = activitats.lloc.str.replace(",", ";")
activitats.preu = activitats.preu.str.replace(",", ";")

* Creació de noves variables

Preu_mitjà

In [201]:
activitats["preu_mitja"] = activitats.preu.str.findall(r"\d+[,'.]?\d*")
activitats["preu_mitja"].loc[activitats.preu.notnull()] = activitats.loc[activitats.preu.notnull()].preu_mitja.apply(lambda x: np.float64(np.mean([float(num.replace("'", ".").replace(",", ".")) for num in x])))

C:\Users\quimm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\quimm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\quimm\AppData\Local\Temp\ipykernel_9412\3214746594.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical 

In [202]:
#type(activitats["preu_mitja"].loc[activitats.preu.notnull()][0])

In [203]:
# Modifiquem el . per , per problemes de codificacio en els analistes
#activitats["preu_mitja"] = activitats["preu_mitja"].apply(lambda num: str(num).replace(".", ","))

Categoria New

In [204]:
def assign_category(category_alternative):

    if type(category_alternative) != str:
        category_new = np.nan
    else:
        category_alternative = category_alternative.lower()
        if "fall" in category_alternative:
            category_new = "Falles"
        elif "màg" in category_alternative:
            category_new = "Màgia"
        elif "visita" in category_alternative:
            category_new = "Activitats sobre patrimoni"
        elif "circ" in category_alternative:
            category_new = "Circ"
        elif "concu" in category_alternative:
            category_new = "Concurs"
        elif "premi" in category_alternative:
            category_new = "Premis"
        elif "taller" in category_alternative:
            category_new = "Taller"
        elif "espectac" in category_alternative:
            category_new = "Espectacle"
        else:
            category_new = np.nan

    return category_new


In [205]:
activitats["categoria_new"] = activitats["categoria_alternativa"].apply(assign_category)
activitats["categoria_new"] = np.where(activitats['categoria_new'].isnull(), activitats['categoria'], activitats['categoria_new'])

Mes

In [206]:
activitats["Mes"] = pd.to_datetime(activitats.data, format= "%d/%m/%Y").dt.month_name()

Any de l'analisi

In [207]:
def set_analisi_year(date_):
    if date_ < pd.to_datetime("2023/06/30"):
        res = "analisi_2023"
    elif date_ < pd.to_datetime("2024/06/30"):
        res = "analisi_2024"
    elif date_ < pd.to_datetime("2025/06/30"):
        res = "analisi_2025"
    
    else:
        res = "analisi_no_definit"

    return res

In [208]:
pd.to_datetime(activitats.data, format="%d/%m/%Y").describe()

count                            21228
mean     2023-12-15 11:46:34.120972544
min                2022-10-01 00:00:00
25%                2023-01-27 00:00:00
50%                2023-12-14 00:00:00
75%                2024-11-07 00:00:00
max                2026-02-22 00:00:00
Name: data, dtype: object

In [209]:
activitats.loc[pd.to_datetime(activitats.data, format="%d/%m/%Y") > pd.to_datetime("2024/06/30")]

,marca_temps,localitat,data,categoria,categoria_alternativa,nom_activitat,companyia,lloc,llengua_activitat,preu,regidoria,llengua_publicitat,font,persona_dades,preu_mitja,categoria_new,Mes
4612,11/04/2024 13:05:08,Torrevieja,7/12/2024,Música,NaN,Concierto Camela,Camela,"Palacio de Deportes ""Tavi y Carmona""",Espanyol,NaN,Cultura,Espanyol,https://www.facebook.com/photo/?fbid=757277143...,Christian,NaN,Música,December
5807,3/02/2025 10:21:44,"Alfàs del Pi, l'",2/11/2024,Dansa,NaN,Gala Flamenca,Juan Carlos Avecilla,Auditori Casa de la Cultura,Espanyol,20,Cultura,Espanyol,https://www.lalfas.es/concejalias/cultura/este...,Martí,20.0,Dansa,November
5808,3/02/2025 10:23:01,Alcoi,22/11/2024,Teatre,NaN,Betlem de Tirisiti,La dependent,Teatre Principal,Valencià/català,5,En col·laboració o organitzat per una entitat ...,Valencià/català,https://webdetirisiti.com/la-companyia/,Alba,5.0,Teatre,November
5809,3/02/2025 10:25:24,Albatera,23/11/2024,Música,NaN,Concierto de Santa Cecilia por las víctimas y...,asociación musical ‘Santiago Apóstol’ y la pia...,Casa de Cultura ‘Miguel Hernández’ de Albatera,No hi ha informació,NaN,En col·laboració o organitzat per una entitat ...,Espanyol,https://todoculturavegabaja.es/eventos/albater...,Neus,NaN,Música,November
5810,3/02/2025 10:25:40,Altea,4/10/2024,Música,NaN,Concierto del Coro Gospel Altea,Coro Gospel Altea,Parroquia de Santa Anna (Altea la Vella),No hi ha informació,NaN,Cultura,Espanyol,https://www.alteacultural.com/2024/10/04/agend...,Gabriel,NaN,Música,October
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2796,28/03/2025 10:39:29,Vinaròs,20/03/2025,Club de lectura,NaN,Llibre de meravelles,Vicent Andrés Estellés,Sala d'actes Biblioteca Municipal,Valencià/català,NaN,En col·laboració o organitzat per una entitat ...,Valencià/català,https://www.vinaros.es/index.php/es/contenido/...,Gabriel,NaN,Club de lectura,March
2797,28/03/2025 10:41:17,Vinaròs,21/03/2025,Lectura poemes,NaN,II Celebració del Dia Mundial de la Poesía,Eduard Marco i Alfred Giner Sorolla,Sala d'actes Biblioteca Municipal,No hi ha informació,NaN,En col·laboració o organitzat per una entitat ...,Valencià/català,https://www.vinaros.es/index.php/es/contenido/...,Gabriel,NaN,Lectura poemes,March
2798,28/03/2025 10:42:21,Vinaròs,21/03/2025,Cinema,NaN,Las niñas,Pilar Romero,Auditori Vinalab,Espanyol,NaN,En col·laboració o organitzat per una entitat ...,Valencià/català,https://www.vinaros.es/index.php/es/contenido/...,Gabriel,NaN,Cinema,March
2799,28/03/2025 10:43:38,Vinaròs,22/03/2025,Música,NaN,VII Trobada d'oboés de les Terres de l'Ebre i ...,NaN,Auditori Vinalab,"Valencià/català, No hi ha informació",NaN,Cultura,Valencià/català,https://www.vinaros.es/index.php/es/contenido/...,Gabriel,NaN,Música,March


In [ ]:
activitats["any_estudi"] = pd.to_datetime(activitats.data, format="%d/%m/%Y").apply(lambda x: set_analisi_year(x))

* Nou conjunt de dades: activitats_summaries.csv

llengua activitat

In [ ]:
grup1 = activitats.groupby(["localitat","llengua_activitat"])["llengua_activitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(3).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

llengua_activitat_top1 = grup1.groupby("localitat").apply(lambda x : x.head(1).reset_index(drop = True))["llengua_activitat"]
llengua_activitat_top2 = grup1.groupby("localitat").apply(lambda x : x.head(2).tail(1).reset_index(drop = True))["llengua_activitat"]
llengua_activitat_top3 = grup1.groupby("localitat").apply(lambda x : x.tail(1).reset_index(drop = True))["llengua_activitat"]

C:\Users\quimm\AppData\Local\Temp\ipykernel_9412\334360915.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(3).reset_index(drop = True))


llengua publicitat

In [ ]:
grup1 = activitats.groupby(["localitat","llengua_publicitat"])["llengua_publicitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(2).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

llengua_publicitat_top1 = grup1.groupby("localitat").apply(lambda x : x.head(1).reset_index(drop = True))["llengua_activitat"]
llengua_publicitat_top2 = grup1.groupby("localitat").apply(lambda x : x.tail(1).reset_index(drop = True))["llengua_activitat"]

C:\Users\quimm\AppData\Local\Temp\ipykernel_9412\3631808114.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(2).reset_index(drop = True))


Language position

In [ ]:
grup1 = activitats.groupby(["localitat","llengua_activitat"])["llengua_activitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

C:\Users\quimm\AppData\Local\Temp\ipykernel_9412\2141191287.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).reset_index(drop = True))


In [ ]:
pos_val = grup1.groupby("localitat").apply(lambda x : x.index[x["llengua_activitat"]=="Valencià/català"])
pos_val = pd.DataFrame(pos_val)

pos_cas = grup1.groupby("localitat").apply(lambda x : x.index[x["llengua_activitat"]=="Espanyol"])
pos_cas = pd.DataFrame(pos_cas)

In [ ]:
def get_position(array):
    "get the position of the language in the most used ones. If not used, number 5 assigned"
    try:
        return array[0][0][1]
    except:
        return 5

In [ ]:
positions_val = []

for i in range(pos_val.shape[0]):
    positions_val.append(get_position(pos_val.iloc[i]))

In [ ]:
positions_cas = []

for i in range(pos_cas.shape[0]):
    positions_cas.append(get_position(pos_cas.iloc[i]))

N activitats

In [ ]:
grup1 = activitats.groupby(["localitat"])["llengua_activitat"].count()
n_activivitats = grup1.values

* Creating datasets

activitats_summaries

In [ ]:
activitats_summaries = pd.DataFrame(llengua_activitat_top1)
activitats_summaries = activitats_summaries.reset_index()
del activitats_summaries["level_1"]
activitats_summaries.columns = ['localitat', 'llengua_activitat_top1']

In [ ]:
activitats_summaries['llengua_activitat_top2'] = llengua_activitat_top2.values
activitats_summaries['llengua_activitat_top3'] = llengua_activitat_top3.values
activitats_summaries['llengua_publicitat_top1'] = llengua_publicitat_top1.values
activitats_summaries['llengua_publicitat_top2'] = llengua_publicitat_top2.values
activitats_summaries["position_val_in_llengua_activitat"] = positions_val
activitats_summaries["position_esp_in_llengua_activitat"] = positions_cas
activitats_summaries["n_activitats"] = n_activivitats

In [ ]:
#ratio_activitats_nhabitants
aux = activitats_summaries.merge(ajuntaments_info[["ajuntament", "n_habitants"]], how ='left', left_on="localitat", right_on = "ajuntament")
activitats_summaries["ratio_activitats_nhabitants"] = aux["n_activitats"]/(aux["n_habitants"]/1000)

Saving Datasets

In [ ]:
activitats.to_csv("data/activitats.csv", index=False)

In [ ]:
activitats_summaries.to_csv("data/activitats_summaries.csv", index=False)

In [ ]:
ajuntaments_info.to_csv("data/ajuntaments_info.csv", index=False)